In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from api_keys import client_id, client_secret

from keywords import keywords

import pandas as pd
import re

sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=client_id, client_secret=client_secret))

In [ ]:
# Get total number of episodes for the show
show_id = '4rOoJ6Egrf8K2IrywzwOMk'
podcast = sp.show(show_id=show_id, market='US')

total_episodes = podcast['total_episodes']

print(total_episodes)

# Use this cell if you need to get data from API

In [ ]:
# Get data for all shows
showNum = 0
data = []
# Pagination for show requests
while showNum <= total_episodes:
    print("Current Show Count: ", showNum)
    result = sp.show_episodes(show_id=show_id, limit=50, offset=showNum, market='US')

    episodes = result['items']

    # Loop through the 50 episodes requested
    for episode in episodes:
        title = episode['name']
        description = episode['description']
        duration_ms = episode['duration_ms']
        release_date = episode['release_date']
        
        myEpisode = {
            'title' : title,
            'description' : description,
            'duration_ms' : duration_ms,
            'release_date' : release_date
        }
        
        data.append(myEpisode)
    showNum += 50

print(len(data))

In [ ]:
# Create DF from data
df = pd.DataFrame(data)
df.head()

In [ ]:
# Change MS column to Seconds
df['duration_ms'] = df['duration_ms'].apply(lambda x: float(x / 60000))
df.head()

In [ ]:
# Rename Column
df.rename(columns={'duration_ms' : 'duration_seconds'})

In [ ]:
# Split release date into seperate columns
df['release_date'] = df['release_date'].astype('datetime64[ns]')
df['day'] = df['release_date'].map(lambda x: x.day)
df['month'] = df['release_date'].map(lambda x: x.month)
df['year'] = df['release_date'].map(lambda x: x.year)
df.head()

In [ ]:
# Find guest in show
guests = []
for row in df['title']:
    guest = re.findall(r"[A-Z][a-z]+,?\s+(?:[A-Z][a-z]*\.?\s*)?[A-Z][a-z]+", row)
    if guest:
         guests.append(guest[0])
    else:
        guests.append(pd.NaT)
df['guest'] = guests
df.head()

In [ ]:
df['guest'].count

In [ ]:
df.dropna()

# Use these cells if you already have CSV File

In [ ]:
# Pull Shows From CSV
df = pd.read_csv('data_files/podcast_data.csv')
df.head()

In [ ]:
# Save to CSV To Reuse
df.to_csv('data_files/podcast_data.csv', index=False)

In [ ]:
from keywords import keywordsList

In [ ]:
keywords = [
    'comedian',
    'comic',
    "phd",
    'actor',
    'writer',
    'author',
    'ufc',
    'musician',
    'artist',
    'founder',
    'youtuber',
    'podcast',
    'ceo',
    'philanthropist',
    'singer/songwriter',
    'screenwriter',
    'scientist',
    'co-host',
    'journalist',
    'entrepreneur',
    'astrophysicist',
    'politician',
    'pilot'
]

In [ ]:
dataset = []
for desc in df['description']:
    newDesc = desc.lower()
    newDesc = newDesc.replace(',', '')
    newDesc = newDesc.replace('.', '')
    split = set(newDesc.split())
    found = split.intersection(keywords) 
    if found:
        dataset.append(found)
    else:
        dataset.append(pd.NaT)
    
df['category'] = dataset
df.head()

In [ ]:
df = df.dropna()

In [ ]:
df.tail()

In [ ]:
category_subsets = {
    'comedy'   : ['comedian', 'comic', 'stand-up'],
    'science'  : ['phd', 'astrophysicist', 'scientist'],
    'music'    : ['singer/songwriter', 'musician', 'artist'],
    'sports'   : ['ufc', 'nfl', 'mlb', 'nba', 'nhl'],
    'business' : ['founder', 'entrepreneur', 'ceo'],
    'movies'   : ['actor', 'screenwriter'],
    'writer'   : ['author', 'writer'],
}

catList = ['comedy', 'science', 'music', 'sports', 'business', 'movies', 'writer']

In [ ]:
category_counts = {
    'comedy' : 0,
    'science' : 0,
    'music' : 0,
    'sports' : 0,
    'business' : 0,
    'movies' : 0,
    'writer' : 0
}


for myList in df['category']:
    for cat in myList:
        for mainCat in catList:
            if cat in category_subsets[mainCat]:
                category_counts[mainCat] += 1
    
    
print(category_counts)

In [ ]:
comedy_df = pd.DataFrame()
comedy_df['title'] = ''
comedy_df['description'] = ''
comedy_df['duration_ms'] = ''
comedy_df['release_date'] = ''
comedy_df['day'] = ''
comedy_df['month'] = ''
comedy_df['year'] = ''
comedy_df['guest'] = ''
comedy_df['category'] = ''

for i, row in df.iterrows():
    myList = row['category']
    for cat in myList:
        if cat in category_subsets['comedy']:
            frame = row.to_frame()
            comedy_df.merge(frame)
            
comedy_df.tail()
              